The goal is to estimate the value of Y column (n x 1) by $\hat{Y}$, where
\begin{equation}
\hat{Y} = \sum_{i=1}^{p} \beta_{i}*X^{i} + \beta_{0}
\end{equation}
where p is the degree of the polynomial.

### Numerical approach.

In [117]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split


#Mean Squared error.
def MSE(B:np.ndarray, X:np.ndarray, y:np.ndarray, already_norm: bool = False) ->float:
    """"The functions computes the mean square error between BX and Y"""
    Predicted = np.matmul(X, B)


    #Normalize the output predicted values if they aren't yet.
    if already_norm == False:
        Predicted_normalized = MinMaxScaler().fit_transform(Predicted.reshape(-1,1))
        y_normalized = MinMaxScaler().fit_transform(y.reshape(-1,1))


    Error: float = np.sum((Predicted_normalized - y_normalized)**2)

    return Error

#Mean absolute error.
def MAE(B:np.ndarray, X:np.ndarray, y:np.ndarray, already_norm: bool = False) -> float:
    """"The functions computes the mean absolute error between BX and Y"""
    Predicted = np.matmul(X, B)
   
  #Normalize the output predicted values if they aren't yet.
    if already_norm == False:
        Predicted_normalized = MinMaxScaler().fit_transform(Predicted.reshape(-1,1))
        y_normalized = MinMaxScaler().fit_transform(y.reshape(-1,1))


    Error: float = np.sum(np.abs(Predicted_normalized - y_normalized))

    return Error



def R_squared(B:np.ndarray, X:np.ndarray, y:np.ndarray) ->float:
    """"The functions computes the mean absolute error between BX and Y"""

    SS_res = MSE(B, X, y)
    SS_var = np.var(y)

    R = 1 - SS_res/SS_var

    return R


class PolynRegression():
    def __init__(self, Dataset:np.ndarray, tar_var_id:int, predictor_id: int ,p:int, train_size:float, 
                 funkcja_bledu: callable,
                 dx:float = 10**(-8), eps:float =  10**(-8), lr:float =  0.001 ) -> None:
        """Constructor of the Polynomial Regression. p is a fixed degree of the polynomial. Dataset is a dataset the regression is learning from.
        predictor is the id of the predicting feature and tar_var is a predicted feature. Train_size is a fraction of the Dataset used for learning. 0 < train_size <=1"""
        #id of the targer variable
        self.tar_var_id = tar_var_id
        #id of the predictor
        self.predictor_id = predictor_id

        #Consider only the dataset with predictor and target variable.
        self.Dataset = Dataset[:, [self.predictor_id, self.tar_var_id]]
        self.n = Dataset.shape[0]

        #The error function.
        self.funkcja_bledu = funkcja_bledu
        #the degree of the polynomial.
        self.p = p
        #the size of training dataset.
        self.train_size = train_size
        #the size of testing dataset.
        self.test_size = 1 - train_size
        #the offset for derivatives.
        self.dx = dx
        #the division-by-zero preventing value.
        self.eps = eps
        #the learning rate.
        self.lr = lr

        self.PolynomializeFeature()


    
    def PolynomializeFeature(self,):
        #Create different variables being the consecutive degrees of X variable (X^0, X^1, ..., X^p)
        self.PolyPredictor = np.column_stack([self.Dataset[:, 0]**i for i in range(0, self.p+1)])
        self.tar_var = self.Dataset[:, -1]


        self.PolDataset = np.column_stack((self.PolyPredictor, self.tar_var))

        self.train_set, self.test_set = train_test_split(self.PolDataset, train_size = self.train_size)
        
        #Extract the predicting variable from the training set.
        self.train_predictor = self.train_set[:, :self.p+1]
        #Extract the target variable from the training set.
        self.train_tarvar = self.train_set[:, -1]

        #Extract the predicting variable from the testing set.
        self.test_predictor = self.test_set[:, :self.p+1]
        #Extract the target variable from the testing set.
        self.test_tarvar = self.test_set[:, -1]

    
       
    
        

    def ComputeGradient(self, B:np.ndarray):
        """Computes the Gradient at point B"""
        Gradient = np.zeros(shape = [self.p+1], dtype = np.float64)



        odejmik = self.funkcja_bledu(B, self.train_predictor, self.train_tarvar)
    

        for i in range(self.p+1):
            B_copy = B.copy()
            B_copy[i] += self.dx

            Gradient[i] = (self.funkcja_bledu(B_copy, self.train_predictor, self.train_tarvar) - odejmik )/self.dx
        
        return Gradient
    
    def UpdateParameters(self, params, b1:float=0.4, b2:float =0.4) -> np.ndarray:
        #Define the first moment and the second moment of the gradient
        m1 = np.zeros(shape = [self.p+1])
        m2 = np.zeros(shape = [self.p+1])

        Gradient = self.ComputeGradient(params)

        #Update the moments.
        m1 = (b1*m1 + (1-b1)*Gradient)/(1-b1)
        m2 = (b2*m2 + (1-b2)*Gradient**2)/(1-b2)

        #Update the parameters

        new_parameters = params - self.lr * m1/(np.sqrt(m2)+self.eps)
    
        return new_parameters


    def FitTheParameters(self,n_repeat:int):
        self.Parameters = np.ones(shape = [self.p+1], dtype = np.float64)


        for i in range(n_repeat):
            self.Parameters = self.UpdateParameters(self.Parameters)

        return self.Parameters
    
    def Score(self,):
        return self.funkcja_bledu(self.Parameters, self.test_predictor, self.test_tarvar)
    
    

    

    

np.random.seed(1)
Dataset = np.random.uniform(-3, 3, size = [5000,5])




regresja = PolynRegression(Dataset, tar_var_id = 2, predictor_id = 0, p = 3, train_size = 0.8, funkcja_bledu = MSE)
print(regresja.FitTheParameters(500))
print(regresja.Score())



[1.00798814 0.5        0.5        1.5       ]
116.42872893813495
